# INSTALL AND IMPORT NECESSARY MODULE, LIBRARY

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install simpletransformers

In [ ]:
import logging
import pandas as pd
from simpletransformers.ner import NERModel, NERArgs
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# READ DATA

In [ ]:
train_data = pd.read_csv("/content/drive/MyDrive/NHANDANG/DATA/FULL/full_TRAIN.csv")
eval_data = pd.read_csv("/content/drive/MyDrive/NHANDANG/DATA/FULL/full_DEV.csv")
test_data = pd.read_csv("/content/drive/MyDrive/NHANDANG/DATA/FULL/full_TEST.csv")

# PRECESSING FOR FINE-TUNE

In [ ]:
train_data['words'] = train_data['words'].astype(str)
train_data['labels'] = train_data['labels'].astype(str)
eval_data['words'] = eval_data['words'].astype(str)
eval_data['labels'] = eval_data['labels'].astype(str)

In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307476 entries, 0 to 307475
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   sentence_id  307476 non-null  int64 
 1   words        307476 non-null  object
 2   labels       307476 non-null  object
dtypes: int64(1), object(2)
memory usage: 7.0+ MB


In [ ]:
train_data=train_data.dropna()

In [ ]:
train_data.head()

,sentence_id,words,labels
0,0,Pin,B-BATTERY-POSITIVE
1,0,Sài,I-BATTERY-POSITIVE
2,0,tầm,I-BATTERY-POSITIVE
3,0,50h,I-BATTERY-POSITIVE
4,0,cho,I-BATTERY-POSITIVE


In [ ]:
unique_train = set(train_data["labels"])
unique_eval = set(eval_data["labels"])

different_labels = unique_train.difference(unique_eval)
print("Các nhãn khác nhau giữa tập huấn luyện và tập đánh giá:")
print(different_labels)

Các nhãn khác nhau giữa tập huấn luyện và tập đánh giá:
set()


In [ ]:
len(set(eval_data["labels"]))

61

In [ ]:
len(set(train_data["labels"]))

61

In [ ]:
label_counts = train_data['labels'].value_counts()
label_counts

O                         107819
I-PERFORMANCE-NEGATIVE     20800
I-GENERAL-POSITIVE         19711
I-FEATURES-NEGATIVE        18643
I-PERFORMANCE-POSITIVE     16412
                           ...  
B-DESIGN-NEUTRAL              46
B-SCREEN-NEUTRAL              39
B-SER&ACC-NEUTRAL             33
B-STORAGE-NEGATIVE            17
B-STORAGE-NEUTRAL             17
Name: labels, Length: 61, dtype: int64

In [ ]:
label_counts = eval_data['labels'].value_counts()
label_counts

O                         30929
I-GENERAL-POSITIVE         6112
I-FEATURES-NEGATIVE        6086
I-PERFORMANCE-NEGATIVE     5803
I-PERFORMANCE-POSITIVE     4819
                          ...  
B-STORAGE-POSITIVE           12
B-STORAGE-NEGATIVE           12
B-SCREEN-NEUTRAL              9
B-STORAGE-NEUTRAL             7
B-SER&ACC-NEUTRAL             3
Name: labels, Length: 61, dtype: int64

# MODEL

## Xlm - roberta

**1e-4**

In [ ]:
# Get unique labels from the label column in train_df
label_list = train_data["labels"].unique().tolist()

# Configure the model
model_args = NERArgs()
model_args.evaluate_during_training = True
model_args.num_train_epochs = 40
model_args.learning_rate = 1e-4
model_args.best_model_dir="/content/drive/MyDrive/full_xml/1e-4"
model_args.overwrite_output_dir =True
model_args.train_batch_size = 32
model_args.eval_batch_size = 32
model_args.use_cached_eval_features = False
model_args.use_multiprocessing = False
model_args.reprocess_input_data = True
model_args.use_early_stopping = True
model_args.early_stopping_metric = "eval_loss"
model_args.early_stopping_metric_minimize = True
model_args.early_stopping_patience = 3
model_args.early_stopping_consider_epochs=True
model_args.evaluate_during_training=True
# Initialize the model
model = NERModel(
    "xlmroberta",
    "xlm-roberta-base",
    labels=label_list,
    args=model_args
)

# Train the model
# Train the model
model.train_model(train_data, eval_data=eval_data)



result_01, model_outputs, preds_list = model.eval_model(test_data)

In [ ]:
result_01